# Use the langgraph-sdk to interact with your agent's API

In [7]:
from langgraph_sdk import get_client

In [8]:
client = get_client()

In [10]:
assistants = await client.assistants.search()
assistants

[{'assistant_id': '8a0502dc-fdea-56eb-8cf3-a81bdeabd000',
  'graph_id': 'storm',
  'config': {},
  'created_at': '2024-05-07T18:38:45.385890+00:00',
  'updated_at': '2024-05-07T18:38:45.385890+00:00',
  'metadata': {'created_by': 'system'}},
 {'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'graph_id': 'agent',
  'config': {},
  'created_at': '2024-05-07T18:38:44.541342+00:00',
  'updated_at': '2024-05-07T18:38:44.541342+00:00',
  'metadata': {'created_by': 'system'}}]

In [15]:
assistant = next(a for a in assistants if a['graph_id'] == 'agent')
assistant

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'created_at': '2024-05-07T18:38:44.541342+00:00',
 'updated_at': '2024-05-07T18:38:44.541342+00:00',
 'metadata': {'created_by': 'system'}}

In [11]:
thread = await client.threads.create()
thread

{'thread_id': 'e13ecf9e-c542-467b-bb0d-31f9179509cb',
 'created_at': '2024-05-07T18:40:40.494347+00:00',
 'updated_at': '2024-05-07T18:40:40.494347+00:00',
 'metadata': {}}

In [12]:
runs = await client.runs.list(thread['thread_id'])
runs

[]

In [16]:
input = {"messages": [{"role": "human", "content": "whats the weather in la"}]}
async for chunk in client.runs.stream(thread['thread_id'], assistant['assistant_id'], input=input):
    print(chunk)

StreamPart(event='metadata', data={'run_id': 'ab2ce012-afb7-405f-b294-d30dd33caed8'})
StreamPart(event='data', data={'messages': [{'role': 'human', 'content': 'whats the weather in la'}]})
StreamPart(event='data', data={'messages': [{'content': 'whats the weather in la', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': '5d6c2466-6eed-4975-a077-593c0e40e764', 'example': False}, {'content': [{'id': 'toolu_017Q77PE4PoBdPC1aausse4o', 'input': {'query': 'weather in los angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': 'run-da05e793-3c7e-4196-a317-3e9a7c149514', 'example': False, 'tool_calls': [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in los angeles'}, 'id': 'toolu_017Q77PE4PoBdPC1aausse4o'}], 'invalid_tool_calls': []}]})
StreamPart(event='data', data={'messages': [{'content': 'whats the weather in la', 'additional_kwargs': {}

In [17]:
input = {"messages": [{"role": "human", "content": "and in sf"}]}
run = await client.runs.create(thread['thread_id'], assistant["assistant_id"], input=input)
run

{'run_id': '7cba0fd3-e5f0-4a67-8279-adcc91998ce8',
 'thread_id': 'e13ecf9e-c542-467b-bb0d-31f9179509cb',
 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'created_at': '2024-05-07T18:43:02.068807+00:00',
 'updated_at': '2024-05-07T18:43:02.068807+00:00',
 'status': 'pending',
 'metadata': {}}

In [18]:
await client.runs.get(thread['thread_id'], run['run_id'])

{'run_id': '7cba0fd3-e5f0-4a67-8279-adcc91998ce8',
 'thread_id': 'e13ecf9e-c542-467b-bb0d-31f9179509cb',
 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'created_at': '2024-05-07T18:43:02.068807+00:00',
 'updated_at': '2024-05-07T18:43:02.068807+00:00',
 'status': 'pending',
 'metadata': {}}

In [19]:
await client.runs.list_events(thread['thread_id'], run['run_id'])

[{'event_id': 'de18e036-01e4-43e4-a971-b91c5955ba3f',
  'run_id': '7cba0fd3-e5f0-4a67-8279-adcc91998ce8',
  'received_at': '2024-05-07T18:43:04.263106+00:00',
  'span_id': 'eb24a21e-9dfb-4ba1-8af4-7575fd60181d',
  'event': 'on_tool_start',
  'name': 'tavily_search_results_json',
  'data': {'input': {'query': 'weather in san francisco'}},
  'metadata': {'graph_id': 'agent',
   'thread_id': 'e13ecf9e-c542-467b-bb0d-31f9179509cb',
   'created_by': 'system',
   'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca'},
  'tags': ['seq:step:1']},
 {'event_id': '1f4e0da0-108e-43ee-bdb1-199a937f3526',
  'run_id': '7cba0fd3-e5f0-4a67-8279-adcc91998ce8',
  'received_at': '2024-05-07T18:43:04.260646+00:00',
  'span_id': '93035172-3769-497a-9eab-30a6982fa819',
  'event': 'on_chain_start',
  'name': 'action',
  'data': {},
  'metadata': {'graph_id': 'agent',
   'thread_id': 'e13ecf9e-c542-467b-bb0d-31f9179509cb',
   'created_by': 'system',
   'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca'},
